In [1]:
from RPA.Windows import Windows
from time import sleep
from datetime import datetime
from RPA.Desktop import Desktop
import gspread as gs
from tasks.crear_factura_ER import * 

win = Windows()
desk = Desktop()
gc = gs.service_account(filename='tdpcorp-139fbe6dbc3c.json')

In [2]:
data = leer_hoja()
console.print(data)
formulario_factura_de_proveedores()
llenar_datos_factura(data)


[
    'P20418463644',
    '27.04.2023',
    '01.05.2023',
    '01.05.2023',
    'F834',
    '11732',
    'OPER',
    'THERM',
    'LURIN',
    'DELIVERY',
    '9263111',
    '8.03',
    '0',
    '9.8',
    'RECARGO POR DELIVERY',
    '1011116-23015',
    '0'
]

Formulario de factura de proveedores abierto

ValueError: could not convert string to float: ''

In [ ]:
worksheet = workbook.get_worksheet_by_id(0)
data = worksheet.row_values(2)
data